In [11]:
import numpy as np
import time

import PIL.Image as Image
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

import datetime

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [2]:
mobilenet_v2 ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"
IMAGE_SHAPE = (224, 224)

In [3]:
labels_path = tf.keras.utils.get_file('ImageNetLabels.txt','https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
imagenet_labels = np.array(open(labels_path).read().splitlines())

In [2]:
batch_size = 32
img_height = 224
img_width = 224
data_dir = '../low-resolution'
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  shuffle = True  
)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  shuffle = True  
    
)

Found 70432 files belonging to 130 classes.
Using 56346 files for training.


2024-03-01 19:11:09.536693: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-01 19:11:09.586801: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-01 19:11:09.587046: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Found 70432 files belonging to 130 classes.
Using 14086 files for validation.


In [10]:
class_names = np.array(train_ds.class_names)


In [7]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [7]:
feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(224, 224, 3), trainable=False)

In [8]:

normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.
     

In [9]:
from tensorflow import keras

In [10]:
dog_classes_count = len(class_names)

model = tf.keras.Sequential([
      keras.layers.RandomFlip("horizontal",
                      input_shape=(img_height,
                                  img_width,
                                  3)),
  keras.layers.RandomRotation(0.1),
  keras.layers.RandomZoom(0.1),
  pretrained_model_without_top_layer,
  tf.keras.layers.Dense(dog_classes_count)
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_flip (RandomFlip)    (None, 224, 224, 3)       0         
                                                                 
 random_rotation (RandomRot  (None, 224, 224, 3)       0         
 ation)                                                          
                                                                 
 random_zoom (RandomZoom)    (None, 224, 224, 3)       0         
                                                                 
 keras_layer_1 (KerasLayer)  (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 130)               166530    
                                                                 
Total params: 2424514 (9.25 MB)
Trainable params: 166530 (650.51 KB)
Non-trainable params: 2257984 (8.61 MB)
_____________

In [11]:

model.compile(
  optimizer="adam",
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])


In [12]:
epochs=20
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
  batch_size=32
)

Epoch 1/20


2024-03-01 14:11:00.240171: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-03-01 14:11:00.604360: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-01 14:11:06.082610: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f829aba7010 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-01 14:11:06.082646: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce MX150, Compute Capability 6.1
2024-03-01 14:11:06.106390: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1709295066.260991    4066 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1761/1761 [==============================] - 477s 266ms/step - loss: 1.2282 - acc: 0.6561 - val_loss: 0.8356 - val_acc: 0.7435
Epoch 2/20
1761/1761 [==============================] - 456s 259ms/step - loss: 0.9404 - acc: 0.7187 - val_loss: 0.8434 - val_acc: 0.7417
Epoch 3/20
1761/1761 [==============================] - 456s 259ms/step - loss: 0.8642 - acc: 0.7372 - val_loss: 0.8085 - val_acc: 0.7532
Epoch 4/20
1761/1761 [==============================] - 455s 258ms/step - loss: 0.8195 - acc: 0.7454 - val_loss: 0.8225 - val_acc: 0.7516
Epoch 5/20
1761/1761 [==============================] - 455s 258ms/step - loss: 0.7875 - acc: 0.7536 - val_loss: 0.8507 - val_acc: 0.7436
Epoch 6/20
1761/1761 [==============================] - 456s 259ms/step - loss: 0.7589 - acc: 0.7627 - val_loss: 0.8229 - val_acc: 0.7462
Epoch 7/20
1761/1761 [==============================] - 456s 259ms/step - loss: 0.7431 - acc: 0.7649 - val_loss: 0.8545 - val_acc: 0.7513
Epoch 8/20
1761/1761 [=======================

In [20]:
test_ds= tf.keras.utils.image_dataset_from_directory(
  'archive/cropped/test',
  image_size=(img_height, img_width),
  batch_size=batch_size)


Found 8580 files belonging to 120 classes.


In [46]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [47]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [21]:
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.

In [22]:
model.evaluate(test_ds)


269/269 [==============================] - 55s 206ms/step - loss: 0.5655 - acc: 0.8244


[0.5654774308204651, 0.8243589997291565]

In [ ]:
model.save('saved_model/2')

In [7]:
from tensorflow.keras.models import load_model

In [8]:
trans_model = load_model('saved_model/2')

In [4]:
grace_hopper = np.array(grace_hopper)/255.0
import sys
np.set_printoptions(threshold=sys.maxsize)
array = grace_hopper[np.newaxis, ...]
import json
y = json.dumps(array.tolist(),separators=(',', ':'), 
          sort_keys=True, 
          indent=4)

f = open("demofile3.txt", "w")
f.write(y)
f.close()

In [66]:
sunflower_url = "https://i.ytimg.com/vi/-ylolmt2e6o/hq720.jpg?sqp=-oaymwEhCK4FEIIDSFryq4qpAxMIARUAAAAAGAElAADIQj0AgKJD&rs=AOn4CLA58XAExELq9exmSeiwGNYiy5ZjhA"
sunflower_path = tf.keras.utils.get_file('a', origin=sunflower_url)
img = tf.keras.utils.load_img(sunflower_path, target_size=(224, 224))
img = np.array(img)/255.0

img_array = tf.expand_dims(img, 0) 
predicted_img(img_array)

1/1 [==============================] - 0s 36ms/step


('n02106662-German_shepherd', 99.56305027008057)

In [20]:
img = Image.open('../image-test/airedale.jpeg').resize((224,224))
img = np.array(img)/ 255

img = tf.expand_dims(img, 0) 
predicted_img(img)

1/1 [==============================] - 0s 25ms/step


('200-n000008-Airedale', 87.72645592689514)

In [12]:
def predicted_img(img_array):
    predicted = trans_model.predict(img_array)
    score = tf.nn.softmax(predicted[0])
    predicted = np.argmax(predicted, axis=1)
    return class_names[predicted][0] , 100 * np.max(score)

In [69]:
plt.figure(figsize=(10, 5))
for images, labels in train_ds.take(1):
  for i in range(8):
    predicted = model.predict((np.array(images[i]))[np.newaxis, ...])

    predicted = np.argmax(predicted, axis=1)
    actaul = class_names[labels[i]]
    pred =  class_names[predicted][0] 
    print(actaul +'--predicted:\n' + pred)
   

1/1 [==============================] - 0s 26ms/step
n02085620-Chihuahua--predicted:
n02085620-Chihuahua
1/1 [==============================] - 0s 22ms/step
n02092002-Scottish_deerhound--predicted:
n02092002-Scottish_deerhound
1/1 [==============================] - 0s 23ms/step
n02107683-Bernese_mountain_dog--predicted:
n02107683-Bernese_mountain_dog
1/1 [==============================] - 0s 25ms/step
n02092339-Weimaraner--predicted:
n02092339-Weimaraner
1/1 [==============================] - 0s 23ms/step
n02098413-Lhasa--predicted:
n02098413-Lhasa
1/1 [==============================] - 0s 23ms/step
n02101388-Brittany_spaniel--predicted:
n02101388-Brittany_spaniel
1/1 [==============================] - 0s 23ms/step
n02090622-borzoi--predicted:
n02090622-borzoi
1/1 [==============================] - 0s 21ms/step
n02096294-Australian_terrier--predicted:
n02096294-Australian_terrier


<Figure size 1000x500 with 0 Axes>